In [100]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import classification_report
import pickle

In [101]:
df= pd.read_csv('data_B.csv')

In [102]:
df.head()

,Unnamed: 0,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,churn
0,0,37969,15780804,Ugorji,850.0,France,Male,42.0,5,0.00,2,1,1,105560.01,0
1,1,121427,15593974,Dickson,597.0,Germany,Male,46.0,9,112291.06,2,1,1,47410.06,0
2,2,151801,15655274,Teng,637.0,Germany,Male,35.0,3,145974.79,2,1,0,180934.67,0
3,3,149981,15645404,McWilliams,695.0,France,Male,39.0,8,0.00,1,1,0,162669.73,0
4,4,115198,15625881,Yobachi,695.0,France,Female,29.0,0,109109.89,1,1,0,120770.80,0


In [103]:
df.columns

Index(['Unnamed: 0', 'id', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'churn'],
      dtype='object')

In [104]:
df['NumOfProducts'].unique()

array([2, 1, 3, 4])

In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41259 entries, 0 to 41258
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       41259 non-null  int64  
 1   id               41259 non-null  int64  
 2   CustomerId       41259 non-null  int64  
 3   Surname          41259 non-null  object 
 4   CreditScore      41251 non-null  float64
 5   Geography        41259 non-null  object 
 6   Gender           41259 non-null  object 
 7   Age              41259 non-null  float64
 8   Tenure           41259 non-null  int64  
 9   Balance          41259 non-null  float64
 10  NumOfProducts    41259 non-null  int64  
 11  HasCrCard        41259 non-null  int64  
 12  IsActiveMember   41259 non-null  int64  
 13  EstimatedSalary  41259 non-null  float64
 14  churn            41259 non-null  int64  
dtypes: float64(4), int64(8), object(3)
memory usage: 4.7+ MB


In [106]:
df.isnull().sum()

Unnamed: 0         0
id                 0
CustomerId         0
Surname            0
CreditScore        8
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
churn              0
dtype: int64

In [107]:
df.duplicated().sum()

0

In [108]:
numeric = []
categorical = []

for i in df.columns:
    if df[i].dtype == 'object':
        categorical.append(i)
    else:
        numeric.append(i)

print(numeric)
print("---")
print(categorical)

['Unnamed: 0', 'id', 'CustomerId', 'CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'churn']
---
['Surname', 'Geography', 'Gender']


In [109]:
categorical.append('churn')
numeric.remove('churn')

In [110]:
df[numeric].describe()

,Unnamed: 0,id,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
count,41259.000000,41259.000000,4.125900e+04,41251.000000,41259.000000,41259.000000,41259.000000,41259.000000,41259.000000,41259.000000,41259.000000
mean,20629.000000,82837.274510,1.569180e+07,656.576253,38.164570,5.025570,55045.726741,1.552655,0.753581,0.496619,112741.311618
std,11910.591715,47737.174121,7.134071e+04,80.196755,8.893375,2.805195,62703.556191,0.549379,0.430931,0.499995,50460.520109
min,0.000000,0.000000,1.556580e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,11.580000
25%,10314.500000,41289.500000,1.563323e+07,597.000000,32.000000,3.000000,0.000000,1.000000,1.000000,0.000000,74800.990000
50%,20629.000000,83294.000000,1.568995e+07,659.000000,37.000000,5.000000,0.000000,2.000000,1.000000,0.000000,118913.530000
75%,30943.500000,124373.000000,1.575647e+07,710.000000,42.000000,7.000000,119639.315000,2.000000,1.000000,1.000000,155846.700000
max,41258.000000,165028.000000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.000000,1.000000,199992.480000


In [111]:
for i in categorical:
    print(f'{i}: {df[i].unique()}')
    print(df[i].unique())

Surname: ['Ugorji' 'Dickson' 'Teng' ... 'Chase' 'Sells' 'Stehle']
['Ugorji' 'Dickson' 'Teng' ... 'Chase' 'Sells' 'Stehle']
Geography: ['France' 'Germany' 'Spain']
['France' 'Germany' 'Spain']
Gender: ['Male' 'Female']
['Male' 'Female']
churn: [0 1]
[0 1]


In [112]:
df = df.drop(columns = ["Unnamed: 0", "id", "CustomerId", "Surname"])

In [113]:
from sklearn.preprocessing import LabelEncoder
labels = LabelEncoder()
df['Geography'] = labels.fit_transform(df['Geography'])
df['Gender'] = labels.fit_transform(df['Gender'])

In [114]:
input_df = df.drop('churn', axis=1)
output_df = df[['churn']]

In [115]:
x_train, x_test, y_train, y_test = train_test_split(input_df, output_df, test_size = 0.2, random_state = 42)

In [116]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33007 entries, 23629 to 15795
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      33000 non-null  float64
 1   Geography        33007 non-null  int64  
 2   Gender           33007 non-null  int64  
 3   Age              33007 non-null  float64
 4   Tenure           33007 non-null  int64  
 5   Balance          33007 non-null  float64
 6   NumOfProducts    33007 non-null  int64  
 7   HasCrCard        33007 non-null  int64  
 8   IsActiveMember   33007 non-null  int64  
 9   EstimatedSalary  33007 non-null  float64
dtypes: float64(4), int64(6)
memory usage: 2.8 MB


In [117]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(x_train), columns=x_train.columns)
X_test  = pd.DataFrame(scaler.fit_transform(x_test), columns=x_test.columns)

In [118]:
X_train

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,-0.136167,1.664445,-1.142418,0.208011,1.061940,1.862077,-1.005502,0.575076,-0.991943,0.174198
1,0.237467,-0.790955,-1.142418,-1.928561,1.775166,0.388511,-1.005502,-1.738900,1.008122,0.967169
2,0.660919,-0.790955,-1.142418,1.669876,1.061940,-0.876797,-1.005502,-1.738900,-0.991943,1.110686
3,-0.547165,0.436745,0.875336,0.770267,0.705328,0.794470,0.814149,0.575076,-0.991943,-0.647077
4,-0.846072,1.664445,0.875336,2.794388,-0.721123,-0.876797,0.814149,0.575076,1.008122,-0.692968
...,...,...,...,...,...,...,...,...,...,...
33002,0.237467,-0.790955,0.875336,0.208011,1.061940,-0.876797,0.814149,0.575076,-0.991943,-0.523492
33003,0.050650,-0.790955,-1.142418,-0.466696,1.061940,-0.876797,0.814149,0.575076,-0.991943,1.442643
33004,-1.692976,-0.790955,0.875336,0.995169,0.705328,-0.876797,2.633800,-1.738900,-0.991943,-0.584466
33005,0.710737,-0.790955,0.875336,3.244193,-1.077736,-0.876797,-1.005502,-1.738900,1.008122,-0.517084


In [119]:
y_train

,churn
23629,0
6996,0
26510,1
40020,1
17668,0
...,...
6265,0
11284,0
38158,1
860,0


In [120]:
import warnings
warnings.filterwarnings('ignore')

RF_class = RandomForestClassifier(criterion= 'gini',max_depth=4)

In [121]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test = imputer.fit_transform(X_test)

In [ ]:
RF_class.fit(X_train, y_train)

In [ ]:
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train, y_train)

In [ ]:
RF_pred = RF_class.predict(X_test)
xgb_pred = xgb_model.predict(X_test)

In [ ]:
print("Random Forest Classification Report:")
print(classification_report(y_test, RF_pred))

print("XGBoost Classification Report:")
print(classification_report(y_test, xgb_pred))

In [ ]:
from sklearn.metrics import accuracy_score
best_model = RF_class if accuracy_score(y_test, RF_pred) > accuracy_score(y_test, xgb_pred) else xgb_model

In [ ]:
with open('best_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)